# PV

## Load modules

In [1]:
import numpy as np
import pandas as pd
import logging
from datetime import datetime

## PV

In [2]:
class MyPV:
    def __init__(self, excel_file : str = './INFO.xlsx', fillNa = 0):
        # 엑셀파일 읽기
        sht_rate = pd.read_excel(excel_file, sheet_name="위험률", header=1)
        sht_code = pd.read_excel(excel_file, sheet_name="코드", header=1)
        sht_expense = pd.read_excel(excel_file, sheet_name="사업비", header=1)
        
        # 전처리
        self.fillNa = fillNa        
        self.sht_rate = sht_rate[['RiskKey', 'x', 'Male', 'Female']].fillna(self.fillNa)
        self.sht_code = sht_code[['KEY', 'BenefitNum', 'ExitCode', 'NonCov',\
            'BenefitCode', 'DefryRate', 'ReducRate', 'ReducPeriod', \
                'GrantCode', 'InvalidPeriod']].fillna(self.fillNa)
        self.sht_expense = sht_expense[['KEY', 'sex', 'x', 'n', 'm', 'mPrime',\
            'AMT', 'S', 'alpha1', 'alpha2', 'beta1', 'beta2','betaPrime', \
                'beta5', 'ce', 'gamma']].fillna(self.fillNa)

        # 공시이율
        self.i = 0.0225
        self.v = 1/(1+self.i)

        # 계약 시점에 가입자 수
        self.l0 = 100000

    def setArgs(self, KEY : str, sex : int, x : int,\
        n : int, m : int, mPrime : int, AMT : int, S : float,\
            alpha1 : float, alpha2 : float, beta1 : float, beta2 : float,\
                betaPrime : float, beta5 : float, ce : float, gamma : float):                
        """
        expense sheet의 컬럼명과 동일
        """
        assert m<=n
        assert sex in [1, 2]
        assert x+n<120

        self.KEY = KEY
        self.sex = sex
        self.x = x
        self.n = n
        self.m = m
        self.mPrime = mPrime
        self.AMT = AMT
        self.S = S
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.beta1 = beta1
        self.beta2 = beta2
        self.beta5 = beta5
        self.betaPrime = betaPrime
        self.ce = ce
        self.gamma = gamma
        # 한계연령
        self.w = 108 if self.sex == 1 else 110    
        # projection 기간 (0, 1, 2, ... , n ---> n+1)
        self.proj = self.n+1

    def setArgsIthRow(self, i : int):
        """
        Expense sheet에 i번째 줄의 정보를 읽어서 argument를 setting
        """                
        assert 0<=i and i<self.sht_expense.shape[0]
        row = self.sht_expense.values[i]  
        self.setArgs(*row)

    def getQx(self, riskKey : str, sex : int = None):
        if sex == None:sex = self.sex
        df = self.sht_rate.copy(deep=True)
        df = df.loc[df['RiskKey'] == riskKey]
        if sex==1:df = df[['x', 'Male']]
        else:df = df[['x', 'Female']]
        qx = np.zeros(self.w)
        for row in df.values:
            try:         
                age, rate = row
                qx[int(age)] = rate
                
            except:
                pass
                # self.logger.error(msg=f"{int(age)}세 {riskKey} 위험률  실패")
        return qx[self.x:]

    def Calc(self, returnSample : bool = False):
        df = self.sht_code.copy(deep=True)
        df = df.loc[df['KEY'] == self.KEY]

        # Initialize
        Dx, DxPrime, Nx, NxPrime, tVx, tWx, SUMx  = [0.]*(self.proj), [0.]*(self.proj), [0.]*(self.proj),\
            [0.]*(self.proj), [], [], [0.]*(self.proj)
        sample_dict = {}

        for row in df.values:
            # unpack
            _, benefitNum, exitCode, nonCov, benefitCode, defryRate, reducRate, reducPeriod, grantCode, invalidPeriod = row
            
            ##------------ 위험율 세팅 ------------##           
            if benefitNum != 99:
                # 탈퇴율
                if exitCode == self.fillNa:
                    q_exit = np.zeros(self.proj)
                else:
                    q_exit = self.getQx(exitCode)  
                # 급부율                        
                if benefitCode == self.fillNa:
                    q_benefit = np.zeros(self.proj)
                else:
                    q_benefit = self.getQx(benefitCode)   
            else:
                # 납입면제율           
                if grantCode == self.fillNa:
                    q_grant = np.zeros(self.proj)
                else:
                    q_grant = self.getQx(grantCode)               
            
            ## ------------ lx ------------ ##
            if benefitNum != 99:
                # 유지자                
                lx = [self.l0]   
                for t in range(self.proj-1):
                    q = q_exit[t]
                    if t==0:
                        q*=(1-nonCov/12)
                    l_next = lx[t]*(1-q)
                    lx.append(l_next)   
            else:
                # 납입자
                lx = [self.l0]
                for t in range(self.proj-1):
                    q = q_grant[t]
                    if t==0:
                        q*=(1-invalidPeriod/12)
                    l_next = lx[t]*(1-q)
                    lx.append(l_next)   

            ## ------------ 기수식 ------------ ##
            # Dx, Nx
            if benefitNum == 0:
                Dx = [lx[t]*self.v**t for t in range(self.proj)]
                Nx = [sum(Dx[t:]) for t in range(self.proj)]
            # D'x, N'x
            elif benefitNum == 99:
                DxPrime = [lx[t]*self.v**t for t in range(self.proj)]
                NxPrime = [sum(DxPrime[t:]) for t in range(self.proj)]
            # dx, Cx, Mx, SUMx
            else:
                dx = [q_benefit[t]*lx[t] for t in range(self.proj)]
                Cx = [dx[t]*self.v**(t+0.5) for t in range(self.proj)]
                Mx = [sum(Cx[t:]) for t in range(self.proj)]
                for t in range(self.proj):
                    if t<reducPeriod:
                        sum_x = defryRate*((1-reducRate)*(Mx[t] - Mx[reducPeriod]) + (Mx[reducPeriod] - Mx[self.n]))
                    else:
                        sum_x = defryRate*(Mx[t]-Mx[self.n])
                    SUMx[t] += sum_x

            # sample
            if returnSample:
                if benefitNum == 0:
                    sample_dict['lx'] = lx
                    sample_dict['Dx'] = Dx
                    sample_dict['Nx'] = Dx                    
                elif benefitNum == 99:
                    sample_dict[f"l'x"] = lx
                    sample_dict["D'x"] = DxPrime
                    sample_dict["N'x"] = DxPrime
                else:
                    sample_dict[f'lx({benefitNum})'] = lx
                    sample_dict[f'dx({benefitNum})'] = dx
                    sample_dict[f'Cx({benefitNum})'] = Cx
                    sample_dict[f'Mx({benefitNum})'] = Mx
        if returnSample:
            sample_dict['SUMx'] = SUMx

        ## ------------ 보험료 ------------ ##
        # 월납 납입기수
        Nstar = 12*((NxPrime[0] - NxPrime[self.m]) - 11/24*(DxPrime[0]-DxPrime[self.m]))
        # 월납순보험료
        NP_month = SUMx[0]/Nstar
        # 연납순보험료
        NP_annual = SUMx[0] / (NxPrime[0] - NxPrime[self.m])
        # 기준연납순보험료
        NP_std = SUMx[0]/(NxPrime[0]-NxPrime[min(self.n, 20)])
        # 연납베타순보험료
        NP_beta = NP_annual + self.betaPrime*(Nx[0]-Nx[self.n]-(NxPrime[0]-NxPrime[self.m]))/(NxPrime[0]-NxPrime[self.m])
        # 영업보험료
        G = (NP_month + (self.alpha1 + self.alpha2*NP_std) * Dx[0]/Nstar +\
                self.beta1/12 + self.betaPrime*(Nx[0]-Nx[self.n]-Nstar/12)/Nstar) \
                    /(1-self.beta2-self.beta5)

        ## ------------ 준비금 ------------ ##
        for t in range(self.proj):
            if t==0:
                V=0
            else:
                V = (SUMx[t] + self.betaPrime*(Nx[t] - Nx[self.n]-\
                    (NxPrime[t]-NxPrime[self.m])) - NP_beta *(NxPrime[t]-NxPrime[self.m]))/Dx[t]
            tVx.append(V)        
        ## ------------ 해약환급금 ------------ ##
        # 공제액
        alpha_apply = self.alpha1 + self.alpha2 * NP_std
        alpha_std = NP_std*0.05*min(self.n, 20)+10/1000*self.S
        alpha = min(alpha_apply, alpha_std)
        # 환급금
        for t in range(self.proj):
            m = min(self.m, 7)
            W = max(0, tVx[t] - max(0, (1-t/m)*alpha))
            tWx.append(W)  
        if returnSample:
            sample_dict['tVx'] = tVx
            sample_dict['tWx'] = tWx
            print(f"N* : {Nstar} / NP_month : {NP_month} / NP_beta : {NP_beta} / G : {G}")
            return pd.DataFrame(sample_dict)
        else:
            return {'NP_beta' : NP_beta, 'G' : G, 'tVx' : tVx, 'tWx' : tWx}

In [3]:
pv = MyPV()

In [4]:
pv.sht_rate.head()

,RiskKey,x,Male,Female
0,Qx1,0,0.003839,0.003525
1,Qx1,1,0.000313,0.000298
2,Qx1,2,0.000241,0.000207
3,Qx1,3,0.000168,0.000143
4,Qx1,4,0.000156,0.000130


In [5]:
pv.sht_expense.head()

,KEY,sex,x,n,m,mPrime,AMT,S,alpha1,alpha2,beta1,beta2,betaPrime,beta5,ce,gamma
0,1_1,1,65,10,10,12,100000,0.4,0.004,0.5,0.001,0.065,0.0005,0,0.0,0.0
1,1_1,1,71,10,10,12,100000,0.4,0.004,0.5,0.001,0.065,0.0005,0,0.0,0.0
2,1_1,1,78,10,10,12,100000,0.4,0.004,0.5,0.001,0.065,0.0005,0,0.0,0.0
3,1_1,2,65,10,10,12,100000,0.4,0.004,0.5,0.001,0.065,0.0005,0,0.0,0.0
4,1_1,2,71,10,10,12,100000,0.4,0.004,0.5,0.001,0.065,0.0005,0,0.0,0.0


In [6]:
pv.sht_code.head()

,KEY,BenefitNum,ExitCode,NonCov,BenefitCode,DefryRate,ReducRate,ReducPeriod,GrantCode,InvalidPeriod
0,1_1,0,0,0.0,0,0.0,0.0,0.0,0,0.0
1,1_1,1,Qbj,0.0,Qbj,1.0,0.0,0.0,0,0.0
2,1_1,2,Qc1,0.0,Qc1,0.5,0.0,0.0,0,0.0
3,1_1,3,Qsc,0.0,Qsc,0.5,0.0,0.0,0,0.0
4,1_1,4,Qb,0.0,Qb,0.5,0.0,0.0,0,0.0


In [9]:
pv.setArgsIthRow(0)
pv.Calc(returnSample=True).head(10)

9767606.4725255
NP_month : 0.00043306656181633707 / NP_beta : 0.005137464876140107 / G : 0.0006290447020242198


,lx,Dx,Nx,lx(1),dx(1),Cx(1),Mx(1),lx(2),dx(2),Cx(2),...,lx(5),dx(5),Cx(5),Mx(5),l'x,D'x,N'x,SUMx,tVx,tWx
0,100000.0,100000.000000,100000.000000,100000.000000,235.200000,232.597830,3750.443946,100000.000000,28.300000,27.986899,...,100000.000000,29.200000,28.876942,248.627036,100000.000000,100000.000000,100000.000000,4230.023752,0.000000,0.000000
1,100000.0,97799.511002,97799.511002,99764.800000,265.474133,256.759922,3517.846115,99971.700000,30.991227,29.973937,...,99970.800000,29.591357,28.620018,219.750095,98226.400000,96064.938875,96064.938875,3912.575183,0.002007,0.000000
2,100000.0,95647.443523,95647.443523,99499.325867,296.507991,280.464637,3261.086193,99940.708773,33.580078,31.763139,...,99941.208643,29.482657,27.887419,191.130077,96411.176128,92214.825237,92214.825237,3569.641537,0.003618,0.000000
3,100000.0,93542.732052,93542.732052,99202.817876,328.758138,304.126948,2980.621556,99907.128695,36.166381,33.456726,...,99911.725987,28.874489,26.711157,163.242658,94540.413666,88435.685837,88435.685837,3201.642579,0.004811,0.001071
4,100000.0,91484.334525,91484.334525,98874.059738,361.977933,327.489342,2676.494607,99870.962314,38.550191,34.877200,...,99882.851498,27.867316,25.212169,136.531501,92594.204711,84709.191988,84709.191988,2808.405253,0.005559,0.002754
5,100000.0,89471.231809,89471.231809,98512.081805,395.624521,350.053931,2349.005265,99832.412123,41.130954,36.393225,...,99854.984182,26.761136,23.678615,111.319332,90562.873048,81027.718078,81027.718078,2390.258659,0.005837,0.003967
6,100000.0,87502.427198,87502.427198,98116.457284,427.885870,370.268176,1998.951334,99791.281169,43.708581,37.822928,...,99828.223047,25.156712,21.769193,87.640717,88446.871519,77393.159359,77393.159359,1947.855433,0.005622,0.004687
7,100000.0,85576.945914,85576.945914,97688.571414,457.964023,387.575652,1628.683158,99747.572588,46.282874,39.169267,...,99803.066334,23.353918,19.764456,65.871524,86250.381912,73810.442680,73810.442680,1484.060936,0.004916,0.004916
8,100000.0,83693.834635,83693.834635,97230.607391,485.375192,401.734735,1241.107506,99701.289714,49.053035,40.600155,...,99779.712417,21.053519,17.425551,46.107068,83975.700590,70282.483985,70282.483985,1002.517962,0.003733,0.003733
9,100000.0,81852.161012,81852.161012,96745.232199,511.685533,414.191927,839.372771,99652.236680,51.719511,41.865174,...,99758.658897,18.654869,15.100478,28.681517,81621.441823,66808.913982,66808.913982,507.063000,0.002093,0.002093


In [28]:
rows = []
for i in range(pv.sht_expense.shape[0]):
    row = []
    pv.setArgsIthRow(i)

    calc = pv.Calc()
    NP_beta = calc['NP_beta']
    G = calc['G']
    tVx = calc['tVx']
    tWx = calc['tWx']

    if len(tVx)<11:
        tVx = tVx + [0.]*(11-len(tVx))
        tWx = tWx + [0.]*(11-len(tWx))
    row.append(round(G*100000, 0))
    row.append(round(NP_beta*100000, 0))

    row.append(round(tVx[1]*100000, 0))
    row.append(round(tVx[3]*100000, 0))
    row.append(round(tVx[5]*100000, 0))
    row.append(round(tVx[7]*100000, 0))
    row.append(round(tVx[10]*100000, 0))

    row.append(round(tWx[1]*100000, 0))
    row.append(round(tWx[3]*100000, 0))
    row.append(round(tWx[5]*100000, 0))
    row.append(round(tWx[7]*100000, 0))
    row.append(round(tWx[10]*100000, 0))
    rows.append(row)

result = pd.DataFrame(rows, columns=['G', 'NP_beta', 'V1', 'V3', 'V5', 'V7', 'V10', 'W1', 'W3', 'W5', 'W7', 'W10'])
result.head(10)

,G,NP_beta,V1,V3,V5,V7,V10,W1,W3,W5,W7,W10
0,63.0,514.0,201.0,481.0,584.0,492.0,0.0,0,107.0,397.0,492.0,0.0
1,85.0,740.0,205.0,470.0,542.0,423.0,0.0,0,32.0,323.0,423.0,0.0
2,96.0,841.0,109.0,240.0,282.0,238.0,0.0,0,0.0,48.0,238.0,0.0
3,43.0,311.0,-9.0,-14.0,-7.0,-6.0,0.0,0,0.0,0.0,0.0,0.0
4,41.0,289.0,-1.0,-0.0,4.0,2.0,0.0,0,0.0,0.0,2.0,0.0
5,37.0,245.0,-15.0,-40.0,-49.0,-32.0,0.0,0,0.0,0.0,0.0,0.0
6,91.0,823.0,158.0,326.0,343.0,266.0,0.0,0,70.0,215.0,266.0,0.0
7,92.0,832.0,101.0,235.0,275.0,216.0,0.0,0,0.0,145.0,216.0,0.0
8,37.0,267.0,-4.0,-18.0,-33.0,-37.0,0.0,0,0.0,0.0,0.0,0.0
9,34.0,227.0,-10.0,-12.0,9.0,21.0,0.0,0,0.0,0.0,21.0,0.0
